<a href="https://colab.research.google.com/github/rckormos/Metalprot_learning/blob/main/subsets_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install numpy numba pypivoter
import numpy as np
import numba as nb
from pypivoter.degeneracy_cliques import enumerateCliques

In [33]:
class Subsets:
  def __init__(self, subset_list):
    self.n_subsets = len(subset_list)
    subset_lengths = np.array([len(subset) for subset in subset_list])
    idxs = np.argsort(subset_lengths).flatten()
    self.subsets = np.empty((self.n_subsets, max(subset_lengths)))
    self.subsets[:, :] = np.nan
    for i, idx in enumerate(idxs):
      self.subsets[i, :subset_lengths[idx]] = subset_list[idx]
    self.issubset = np.zeros((self.n_subsets, self.n_subsets), dtype=bool)
    self.delta = np.zeros(self.n_subsets)
    populate_issubset_delta(self.issubset, self.delta,
                            self.n_subsets, self.subsets)

@nb.njit
def populate_issubset_delta(issubset, delta, n_subsets, subsets):
  subset_lengths = (~np.isnan(subsets)).sum(axis=1)
  for i in range(n_subsets):
    for j in range(i, n_subsets):
      flag = True
      for k in subsets[i]:
        if np.isnan(k):
          continue
        if k not in subsets[j]:
          flag = False
          break
      if flag:
        issubset[i, j] = True
        delta[i] += (-1) ** (subset_lengths[j] - subset_lengths[i])

In [6]:
N = 60

In [34]:
# generate random graph
A = np.zeros((N, N))
A[np.triu_indices(N, 1)] = np.random.random(N * (N - 1) // 2).round()
cliques = enumerateCliques(np.argwhere(A))
subsets = sum([[row for row in arr] for arr in cliques if len(arr)], [])
s_cliques = Subsets(subsets)

In [36]:
t = np.random.random(N)
t_sum = np.zeros(N)
for i, subset in enumerate(s_cliques.subsets):
  mask = subset[~np.isnan(subset)].astype(int)
  t_mask = np.zeros(N)
  t_sum[mask] += s_cliques.delta[i] * t[mask]
print(np.abs(t - t_sum).max())

3.630429290524262e-13


In [38]:
delta_sums = (s_cliques.delta * s_cliques.issubset).sum(axis=1)

In [39]:
np.unique(delta_sums)

array([1.])

In [40]:
degeneracy = (A + A.T).sum(axis=0)

In [43]:
t_sum2 = np.zeros(N)
for pair in np.argwhere(A):
  t_mask = np.zeros(N)
  t_sum2[pair] += t[pair] / degeneracy[pair]
print(np.abs(t - t_sum2).max())

7.771561172376096e-16


In [46]:
t_sum3 = np.zeros(N)
degeneracy_all = np.array([len([subset for subset in subsets if i in subset])
                           for i in range(N)])
for clique in subsets:
  t_mask = np.zeros(N)
  t_sum3[clique] += t[clique] / degeneracy_all[clique]
print(np.abs(t - t_sum3).max())

3.863576125695545e-14
